In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from helpers import *
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import cross_validate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
class LogisticModel:
    def __init__(self, **args):
        self.patch_size = 16 # Size of a patch 16x16 pixels
        self.poly_aug = 5 # Degree of the polynomical augmentation
        self.logreg = LogisticRegression(C=1e5, class_weight="balanced")
    
    # Extract 6-dimensional features consisting of average RGB color as well as variance
    def extract_features(self, img):
        feat_m = np.mean(img, axis=(0,1))
        feat_v = np.var(img, axis=(0,1))
        feat = np.append(feat_m, feat_v)
        return feat

    # Extract features for a given image
    def extract_img_features(self, filename):
        img = load_image(filename)
        img_patches = img_crop(img, patch_size, patch_size)
        X = np.asarray([self.extract_features(img_patches[i]) for i in range(len(img_patches))])
        return X
    
    def fit(self, imgs, gt_imgs):
        
        img_patches = crop_imgs(imgs, self.patch_size, self.patch_size)
        gt_patches = crop_imgs(gt_imgs, self.patch_size, self.patch_size)

        X = np.asarray([self.extract_features(img_patches[i]) for i in range(len(img_patches))])
        Y = np.asarray([crop_to_class(gt_patches[i]) for i in range(len(gt_patches))])
        poly = PolynomialFeatures(self.poly_aug, interaction_only=False)
        X = poly.fit_transform(X)
        self.logreg.fit(X, Y)
        
    def predict(self, imgs):
        # If we have a signle image, reshape it
        if len(imgs.shape) < 4:
            imgs = imgs.reshape(1, imgs.shape[0], imgs.shape[1], imgs.shape[2])
        img_patches = crop_imgs(imgs, self.patch_size, self.patch_size)
        X = np.asarray([self.extract_features(img_patches[i]) for i in range(len(img_patches))])
        poly = PolynomialFeatures(self.poly_aug, interaction_only=False)
        X = poly.fit_transform(X)
        return self.logreg.predict(X)
    
    def get_params(self, deep=True):
        return self.logreg.get_params(deep)
    
    def set_params(self, **params):
        return self.logreg.set_params(params)

In [28]:
from cross_validation import k_fold_model
imgs, gt_imgs = load_training('training')
logistic = LogisticModel()
cv_score = k_fold_model(logistic, imgs, gt_imgs)
print("The logistic model scored %.3f ± %.3f as f1-score"%(cv_score['test_f1'].mean(), cv_score['test_f1'].std()))
print("The logistic model scored %.3f ± %.3f as accuracy"%(cv_score['test_acc'].mean(), cv_score['test_acc'].std()))

The dummy model scored 0.579 ± 0.021 as f1-score
The dummy model scored 0.706 ± 0.007 as accuracy


In [24]:
class DummyModel:
    def __init__(self, **args):
        pass
    
    def fit(self, imgs, gt_imgs):
        pass
        
    def predict(self, imgs):
        np.random.seed(42)
        return (np.random.rand((imgs.shape[0]*imgs.shape[1]//16*imgs.shape[2]//16)) > 0.75).astype(int)
    
    def get_params(self, deep=True):
        return {'x': None}
    
    def set_params(self, **params):
        pass

In [26]:
from cross_validation import k_fold_model
imgs, gt_imgs = load_training('training')
dummy = DummyModel()
cv_score = k_fold_model(dummy, imgs, gt_imgs)
print("The dummy model scored %.3f ± %.3f as f1-score"%(cv_score['test_f1'].mean(), cv_score['test_f1'].std()))
print("The dummy model scored %.3f ± %.3f as accuracy"%(cv_score['test_acc'].mean(), cv_score['test_acc'].std()))

The dummy model scored 0.250 ± 0.013 as f1-score
The dummy model scored 0.620 ± 0.011 as accuracy
